In [1]:
# first check the wd
# this should be AIggregation folder, change if its not

import os

print("cwd is", os.getcwd())

if os.path.basename(os.getcwd()) == "notebooks":
    os.chdir("..")
    print("cwd changed to", os.getcwd())

cwd is /home/tmueller/github/AIggregation/notebooks
cwd changed to /home/tmueller/github/AIggregation


In [6]:
# read in datasets

import fiftyone as fo

testdataset = fo.Dataset.from_dir(
    dataset_type=fo.types.YOLOv5Dataset,
    yaml_path = "datasets/tiled_export/tiles/large_slices_large_overlap/data.yaml",
    label_field= "large_slices"
)


dataset_ground = fo.Dataset.from_dir(
    dataset_type=fo.types.YOLOv5Dataset,
    yaml_path = "datasets/groundtruth_testset/data.yaml",
    label_field= "ground_truth"
)


key_fcn = lambda sample: os.path.basename(sample.filepath)

testdataset.merge_samples(dataset_ground, key_fcn=key_fcn)


# view that this worked
session = fo.launch_app(testdataset)




 100% |█████████████████| 100/100 [255.0ms elapsed, 0s remaining, 394.5 samples/s]     
 100% |█████████████████| 165/165 [125.9ms elapsed, 0s remaining, 1.3K samples/s]  
Indexing dataset...
 100% |█████████████████| 100/100 [50.7ms elapsed, 0s remaining, 2.0K samples/s] 
Merging samples...
 165 [282.9ms elapsed, ? remaining, 586.2 samples/s] 


In [ ]:


import scipy
import argparse
import fiftyone as fo
from fiftyone import ViewField as F


dataset = testdataset
prediction = "large_slices"
gt = "ground_truth"
lb = .3
ub = .9


# define a function to calculate the f1 score of a model
def calculate_f1(conf, dataset, prediction, gt):
    conf_view = dataset.filter_labels(prediction, F("confidence") >= conf)
    results = conf_view.evaluate_detections(prediction,
        gt_field= gt,
        eval_key="eval",
        missing="fn")

    fp = sum(conf_view.values("eval_fp"))
    tp = sum(conf_view.values("eval_tp"))
    fn = sum(conf_view.values("eval_fn"))

    f1 = tp/(tp+0.5*(fp+fn))

    return -1.0*f1  #make output negative to use fminbound

# function to find the confidence that results in best f1 score
def optimize_conf(lb, ub, dataset, gt, prediction):

    res = scipy.optimize.fminbound(
                    func=calculate_f1,
                    x1=lb,
                    x2=ub,
                    args=(dataset, prediction, gt),
                    xtol=0.01,
                    full_output=True
    )

    best_conf, f1val, ierr, numfunc = res
    maxf1 = -1.0*f1val
    print("\n \n     best f1          at confidence")
    print(maxf1, best_conf)
    return maxf1, best_conf


# save first output, maxf1, to variable
bestf1 = optimize_conf(lb=lb, ub=ub, dataset=dataset, gt=gt, prediction=prediction)[0]

bestf1




Evaluating detections...
 100% |███████████████████| 95/95 [296.6ms elapsed, 0s remaining, 320.3 samples/s]      
Evaluating detections...
 100% |███████████████████| 90/90 [299.1ms elapsed, 0s remaining, 300.9 samples/s]      
Evaluating detections...
 100% |███████████████████| 88/88 [380.8ms elapsed, 0s remaining, 231.1 samples/s]      
Evaluating detections...
 100% |███████████████████| 89/89 [288.9ms elapsed, 0s remaining, 308.1 samples/s]      
Evaluating detections...
 100% |███████████████████| 93/93 [306.5ms elapsed, 0s remaining, 303.5 samples/s]      
Evaluating detections...
 100% |███████████████████| 91/91 [293.1ms elapsed, 0s remaining, 310.5 samples/s]      
Evaluating detections...
 100% |███████████████████| 90/90 [289.0ms elapsed, 0s remaining, 311.4 samples/s]      
Evaluating detections...
 100% |███████████████████| 90/90 [290.1ms elapsed, 0s remaining, 310.2 samples/s]      
Evaluating detections...
 100% |███████████████████| 90/90 [288.9ms elapsed, 0s remainin

0.7437641723356009

In [ ]:

def optimize_conf(lb, ub, gt, prediction):
   
    best_f1 = -1
    best_threshold = None
    
#https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.fminbound.html

    res = scipy.optimize.fminbound(
                    func=calculate_f1,
                    x1=lb,
                    x2=ub,
                    args=(prediction,gt),
                    xtol=0.01,
                    full_output=True
    )

# the below might need some rescaling
    best_conf, f1val, ierr, numfunc = res
    return -1.0*f1val, best_conf








In [ ]:

# not needed unless I turn this into a script

# parse arguments
if __name__ == "__main__":
    # Initialize parser
    parser = argparse.ArgumentParser()

    parser.add_argument("-gt", default="ground_truth", help = "ground truth detection layer name")
    parser.add_argument("-prediction", default="prediction", help = "ground truth detection layer name")
    parser.add_argument("-dataset", default="dataset", help = "ground truth detection layer name")
    parser.add_argument("-lb", default=".3", help = "lower bound of conf to test")
    parser.add_argument("-ub", default=".9", help = "upper bound of conf to test")
 
 

    parser.add_argument("-ratio", type=float, default=0.8, help = "Train/test split ratio. Dafault: 0.8")

    args = parser.parse_args()

calculate_f1(args.dataset, args.gt, args.prediction, args.lb, args.ub)
optimize_conf(args.lb, args.ub, args.gt, args.predition)